In [19]:
-- List the top five assignments based on total value of donations, categorized by donor type.

WITH donation_details AS (
SELECT 
    d.assignment_id, 
    ROUND(SUM(amount), 2) AS rounded_total_donation_amount, 
    dn.donor_type
FROM donations d
JOIN donors dn ON d.donor_id = dn.donor_id
GROUP BY 
    d.assignment_id, 
    dn.donor_type
)
SELECT 
    a.assignment_name,
    a.region,
    dd.rounded_total_donation_amount,
    dd.donor_type
FROM donation_details dd
JOIN assignments a ON dd.assignment_id = a.assignment_id
ORDER BY dd.rounded_total_donation_amount DESC
LIMIT(5);

,assignment_name,region,rounded_total_donation_amount,donor_type
0,Assignment_3033,East,3840.66,Individual
1,Assignment_300,West,3133.98,Organization
2,Assignment_4114,North,2778.57,Organization
3,Assignment_1765,West,2626.98,Organization
4,Assignment_268,East,2488.69,Individual


In [20]:
-- Identify the assignment with the highest impact score in each region


WITH donation_counts AS (
SELECT 
    assignment_id,
    COUNT(donation_id) AS num_total_donations
FROM donations
GROUP BY assignment_id
),
ranked_assignments AS (
SELECT
    a.assignment_name,
    a.region,
    a.impact_score,
    dc.num_total_donations,
    ROW_NUMBER() OVER(
        PARTITION BY a.region
        ORDER BY a.impact_score DESC
    ) AS rank_in_region
FROM assignments a
JOIN donation_counts dc ON a.assignment_id = dc.assignment_id
WHERE dc.num_total_donations > 0
)
SELECT 
    assignment_name,
    region, 
    impact_score,
    num_total_donations
FROM ranked_assignments
WHERE rank_in_region = 1
ORDER BY region ASC;

,assignment_name,region,impact_score,num_total_donations
0,Assignment_316,East,10.00,2
1,Assignment_2253,North,9.99,1
2,Assignment_3547,South,10.00,1
3,Assignment_2794,West,9.99,2
